In [6]:
import pandas as pd
import numpy as np

In [7]:
from transliterate import translit, get_available_language_codes

In [24]:
pd.options.display.width = None
pd.options.display.max_columns=999

In [8]:
forum_pointz_merged = pd.read_csv('assets/merged/forum_pointz.merged.csv')
sirena = pd.read_csv('assets/Sirena-export-fixed.tab.csv', sep=";")

In [9]:
forum_pointz_merged.drop(['Unnamed: 0'], axis=1, inplace=True)

In [10]:
forum_pointz_merged['date'] = forum_pointz_merged['date'].astype('datetime64[ns]')

In [ ]:
forum_pointz_merged.head()

In [11]:
sirena.drop(['agent_info', 'seat', 'meal'], axis=1, inplace=True)

In [12]:
sirena['depart_date'] = sirena['depart_date'].astype('datetime64[ns]')
sirena['arrival_date'] = sirena['arrival_date'].astype('datetime64[ns]')

In [13]:
sirena['e_ticket'] = sirena['e_ticket'].astype(str)

In [ ]:
sirena.head()

In [28]:
merged = pd.merge(forum_pointz_merged, sirena, how='outer', left_on=['number', 'date', 'flight'], right_on=['card_number', 'depart_date', 'flight'])

In [29]:
merged.drop(['fare_x', 'fare_y', 'depart_time', 'arrival_time'], axis=1, inplace=True)

In [50]:
merged.head()

,sex,first_name,last_name,flight,date,codeshare,country_arrival,city_arrival,airport_arrival,country_departure,city_departure,airport_departure,status,number,birth_date,depart_date,arrival_date,code_share,code,e_ticket,document,class,baggage,additional_info
0,Male,,nan,DL3377,2017-03-04,False,United States of,Pittsburgh PA,PIT,United States of,Boston MA,BOS,Basic,KE 889215424,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Male,,nan,DL3377,2017-03-04,False,United States of,Pittsburgh PA,PIT,United States of,Boston MA,BOS,Elite+,DT 484697244,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Male,,nan,DL3598,2017-07-30,False,United States of,Evansville IN,EVV,United States of,Detroit MI,DTW,Basic,KE 889215424,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Male,,nan,DL3598,2017-07-30,False,United States of,Evansville IN,EVV,United States of,Detroit MI,DTW,Elite+,DT 484697244,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Male,,nan,DL5498,2017-03-31,False,United States of,Birmingham AI,BHM,United States of,New York NY,LGA,Basic,KE 889215424,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
merged['last_name'] = merged['last_name'].fillna('')
merged['first_name'] = merged['first_name'].fillna('')
merged['last_name'] = merged['last_name'].astype(str)
merged['first_name'] = merged['first_name'].astype(str)
merged['name'] = merged['name'].astype(str)

In [36]:
for index, row in merged.iterrows():
    cyr_name = row['name'].split(' ')
    if cyr_name:
        if len(cyr_name) > 0:
            merged.at[index, 'last_name'] = translit(cyr_name[0], 'ru', reversed=True)
        if len(cyr_name) > 1:
            merged.at[index, 'first_name'] = translit(cyr_name[1], 'ru', reversed=True)

In [40]:
merged.drop(['name'], axis=1, inplace=True)

In [43]:
airport_arrival_null = merged.airport_arrival.isnull()
airport_departure_null = merged.airport_departure.isnull()
number_null = merged.number.isnull()

merged.loc[merged[airport_arrival_null].index, 'airport_arrival'] = merged.loc[merged[airport_arrival_null].index, 'dest_code']
merged.loc[merged[airport_departure_null].index, 'airport_departure'] = merged.loc[merged[airport_departure_null].index, 'from_code']
merged.loc[merged[number_null].index, 'number'] = merged.loc[merged[number_null].index, 'card_number']

In [44]:
merged.drop(['dest_code', 'from_code', 'card_number'], axis=1, inplace=True)

In [51]:
date_null = merged.date.isnull()

merged.loc[merged[date_null].index, 'date'] = merged.loc[merged[date_null].index, 'depart_date']

In [52]:
merged.drop(['depart_date'], axis=1, inplace=True)

In [53]:
merged.rename(columns={'date': 'depart_date'}, inplace=True)

In [54]:
merged.to_csv('assets/merged/forum_pointz_sirena.merged.csv')